# Notebook de Generación y predicción
Integrantes:
- Martiñón Luna Jonathan José
- Ortega Ibarra Jaime Jesús
- Tapia López José de Jesús


El presente notebook Genera el modelo de regresión Ridge a nuestro conjunto de entrenamiento, realiza predicción al de validación y finalmente obtiene los valores para el de prueba.

Las mejores variables han sido calculadas con anterioridad, buscando un menor MAE. **En este notebook no se presenta la búsqueda de la mejor combinación.**

-----
# Importando Librerías

In [1]:
#----------------------
# DataFrames
import pandas as pd #Lectura de datos .tsv y creación de archivos .csv
import swifter # Paralelización de funciónes aplicadas (.apply)
#-----------------
# Modelo
from sklearn.linear_model import Ridge # Generación de modelo
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score # Métricas de evaluación
from scipy.stats import pearsonr, spearmanr # Métricas de evaluación
from sklearn import preprocessing # Preprocesamiento
#-----------------
# PLN
from nltk.corpus import stopwords, words # Recuperamos Stopwords y words (Para vocabulario)
from nltk import pos_tag,RegexpTokenizer # Para etiquetar nuestras palabras
from nltk.tokenize import word_tokenize # Tokenización de palabras
import textstat
#-----------------
# Otros
import numpy as np # Operaciones matemáticas y creación de vectores
from collections import Counter # Creación de diccionarios de frecuencias

----
# Generando funciones

In [2]:
def Cuenta_w(Sentence, stop = True):
    '''
    @Descripción:
                Función encargada de tokenizar una oración (STR)
                Y contar la cantidad de stopwords presentes o
                la cantidad de palabras que no son catalogadas
                como stopwords
                
    @Parametros:
        sentence   Oración a contar palabras             STR
        stop       Indica si se desea contar StopWords   BOOL
        
    @Devuelve:
        cant       Cantidad de palabras contadas       INT
    '''
    
    #Tokenizamos y eliminamos signos de puntuación
    tokenized = RegexpTokenizer(r"\w+").tokenize(Sentence) 
    
    #Obtenemos nuestras stopwords
    stopw = stopwords.words('english')
    
    #Podemos elegir contar stop words o palabras que no estén en ellas
    if stop:
        # Capturamos nuestras stop words en el texto
        words = [word for word in tokenized if word.lower() in stopw]
    else:
        # Capturamos nuestras palabras que no son stop words en el texto
        words = [word for word in tokenized if word.lower() not in stopw]
    
    # Obtenemos la cantidad de palabras contadas
    cant = len(words)
    
    #Devolvemos la cantidad
    return cant

In [3]:
def prom_len_words(sentence):
    
    '''
    @Descripción:
                    Calcula la longitud promedio de
                    una palabra dentro de la oración
    @Parámetros:
        sentence    Oración a encontrar promedio    STR
    
    @Devuelve:
        prom        Promedio de longitud de palabra  FLOAT
    
    '''
    
    #Limpiamos para obtener únicamente las palabras.
    tokenized = RegexpTokenizer(r"\w+").tokenize(sentence)
    #Inicializamos nuestra longitud
    long = 0
    #Capturamos la longitud de cada palabra
    for word in tokenized:
        #Incrementamos las longitudes
        long+=len(word)
        
    #Dividimos sobre el total de palabras
    prom = long/len(tokenized)
    
    #Devolvemos el promedio de longitud
    return prom

In [4]:
def contarVocales(oracion):  
    '''
    @Descripción:
                    Función encargada de contar vocales
                    en una oración
    @Parámetros:
        oracion     Oración a contar vocales          STR
        
    @Devuelve:
                    La sumatoria de vocales
    
    '''
    V = ['a','e','i','o','u']
    return sum([1 for i in oracion if i in V])

def contarConsonantes(oracion):  
    '''
    @Descripción:
                    Función encargada de contar consonantes
                    en una oración
    @Parámetros:
        oracion     Oración a contar consonantes          STR
        
    @Devuelve:
                    La sumatoria de consonantes
    '''
    V = ['b','c','d','f','g','h','j','k','l','m'
        'n','p','q','r','s','t','v','w','x','y','z']
    return sum([1 for i in oracion if i in V])

In [5]:
def normalize(data):
    
    '''
    @Descripción:
                Normalización de nuestro conjunto de datos
                
    @Parametros:
        data    Conjunto de datos exclusivamente que van a ser
                usados para realizar la predicción en el modelo      DATAFRAME
                NO target
    
    @Devuelve:
        df      DataFrame de normalizado                  DATAFRAME
    '''
    
    #Capturamos el nombre de las columnas para sustitución posterior
    Cn = data.columns
    #Copiamos el dataframe
    df = data.copy()
    # Obtenemos nuestro dataframe como un arreglo de numpy
    x = df.values
    #Creamos nuestro objeto para escalar
    min_max_scaler = preprocessing.MinMaxScaler()
    #Transformamos nuestros datos a valores entre  0 y 1
    x_scaled = min_max_scaler.fit_transform(x)
    #Almacenamos en un nuevo dataframe y nombramos las columnas como estaban originalmente
    df = pd.DataFrame(x_scaled,columns=Cn)
    return df

In [6]:
def Preprocesa(datos, norm = True):
    '''
    @Descripcion:
                Preprocesamiento del DataFrame para obtener
                las features necesarias para aplicar el modelo
    
    @Parametros:
        datos    DataFrame que debe contener las columnas
                 'sentence' (STR) y 'token' (STR). No importa     DATAFRAME
                 si contiene más.
        norm     Indica si se normalizarán los datos              BOOL
    
    @Devuelve:
        data     DataFrame preprocesado, normalizado y SIN las 
                 features 'sentence' y 'token'
    '''
    
    #Copiamos el dataFrame
    data = datos.copy()
    
    #-----------------------------------
    #Preprocesamiento del Token
    #-----------------------------------
    
    #Contamos la cantidad de Caracteres
    data['Tk_Caracteres'] = data['token'].swifter.apply(len)
    #Contamos la cantidad de Vocales
    data['Tk_Vocales'] = data['token'].swifter.apply(contarVocales)
    #Contamos el número de Consonantes
    data['Tk_Consonantes'] = data['token'].swifter.apply(contarConsonantes)
    # Contamos sílabas en el token
    data['Tk_Silabas'] = data['token'].swifter.apply(textstat.syllable_count)
    # Verificamos si el token es considerado palabra dificil
    data['Tk_Diff_Words'] = data['token'].swifter.apply(textstat.difficult_words)
    
    #-----------------------------------
    #Preprocesamiento de la oración
    #-----------------------------------
    
    # Contamos la cantidad de Caracteres
    data['S_Caracteres'] = data['sentence'].swifter.apply(len)
    # Contamos la cantidad de Vocales
    data['S_Vocales'] = data['sentence'].swifter.apply(contarVocales)
    # Contamos el número de consonantes
    data['S_Consonantes'] = data['sentence'].swifter.apply(contarConsonantes)
    
    # Contamos la proporción de vocales en el token respecto a la oración
    data['Prop_Vocales'] = data['Tk_Vocales'].div(data['S_Vocales'])
    # Contamos la proporción de consonantes en el token respecto a la oración
    data['Prop_Consonantes'] = data['Tk_Consonantes'].div(data['S_Consonantes'])
    # Contamos la proporción de longitud del token respecto a la oración
    data['Prop_len'] = data['Tk_Caracteres'].div(data['S_Caracteres'])
    # Contamos la cantidad de NO'stop_words' en la oración
    data['No_Stop_Words'] = data['sentence'].swifter.apply(Cuenta_w,stop=False)
    # Contamos la cantidad de 'stop_words' en la oración
    data['Stop_Words'] = data['sentence'].swifter.apply(Cuenta_w)
    # Contamos la longitud promedio de una palabra en la oración 
    data['Avg_Mean_Words'] = data['sentence'].swifter.apply(prom_len_words)
    #Contamos sílabas en la oración
    data['Silabas'] = data['sentence'].swifter.apply(textstat.syllable_count)
    #Contamos palabras en la oración SIN SIGNOS DE PUNTUACIÓN
    data['Palabras'] = data['sentence'].swifter.apply(textstat.lexicon_count)
    #Contamos el número de oraciones
    data['#Oraciones'] = data['sentence'].swifter.apply(textstat.sentence_count)
    #Calculamos la facilidad de lectura
    data['Reading_ease'] = data['sentence'].swifter.apply(textstat.flesch_reading_ease)
    # Calculamos otro score de lectura
    data['Kincaid_Lvl'] = data['sentence'].swifter.apply(textstat.flesch_kincaid_grade)
    # calulamos el dale chall readability 
    data['Dale_Chall'] = data['sentence'].swifter.apply(textstat.dale_chall_readability_score)
    
    #--------------------------------------------
    
    #Eliminamos la sentence y el token, pues ya no nos sirven
    data.drop(['sentence','token'],axis=1,inplace=True)
    
    #-------------------------------------------
    
#     #En caso de desear normalizar los datos
#     if norm:
#         # Detectamos si nuestro conjunto cuenta con el TARGET
#         if 'complexity' in data.columns():  
#             #Almacenamos en un auxiliar el TARGET
#             aux = data['complexity']
#             #Normalizamos los datos sin TARGET
#             data = normalize(data.drop(['complexity'],axis=1))
#             #Reunificamos todo
#             data['complexity'] = aux
#         else:
#             #Si no tiene target, podemos normalizar sin problema
#             data = normalize(data)
    
    #Devolvemos el conjunto obtenido
    return data

---
# Lectura de datos

In [7]:
#---------------------
# Train
# --------------------
data_single_train = pd.read_csv('Data/train/lcp_single_train.tsv',sep='\t',keep_default_na=False)
data_multi_train = pd.read_csv('Data/train/lcp_multi_train.tsv',sep='\t',keep_default_na=False)
#---------------------
# Trial
# --------------------
data_single_trial = pd.read_csv('Data/trial/lcp_single_trial.tsv',sep='\t',keep_default_na=False)
data_multi_trial = pd.read_csv('Data/trial/lcp_multi_trial.tsv',sep='\t',keep_default_na=False)
#---------------------
# Test
data_single_test = pd.read_csv('Data/test/lcp_single_test.tsv',sep='\t',keep_default_na=False)
data_multi_test = pd.read_csv('Data/test/lcp_multi_test.tsv',sep='\t',keep_default_na=False)
# --------------------

---
# Preprocesando Los datos

In [8]:
#---------------------
# Train
# --------------------
data_single_train_prep = Preprocesa(data_single_train)
data_multi_train_prep = Preprocesa(data_multi_train)
#---------------------
# Trial
# --------------------
data_single_trial_prep = Preprocesa(data_single_trial)
data_multi_trial_prep = Preprocesa(data_multi_trial)
#---------------------
# Test
data_single_test_prep = Preprocesa(data_single_test)
data_multi_test_prep = Preprocesa(data_multi_test)
# --------------------

---
# Generando el modelo

Dado que anteriormente habíamos realizado combinaciones para obtener el modelo y por lo tanto las características obtenidas en el preprocesamiento han sido exclusivamente las necesarias, simplemente separaremos la complejidad (en el caso de los que tengan)

### Para single

In [9]:
#Partimos
X = data_single_train_prep.drop(['complexity','id','corpus'],axis=1)
y = data_single_train_prep['complexity']

In [10]:
#Generamos el modelo
regression_single = Ridge(alpha=1e-05, normalize=True).fit(X,y)

In [11]:
#Realizamos la predicción
y_pred = regression_single.predict(X)

In [12]:
#Almacenamos el dataset
data_pred = pd.DataFrame({'Y':y,'Y_pred':y_pred})
data_pred.to_csv('Predictions/predictions_data_single_train.csv')

In [13]:
#Calculamos nuestro MAE
print("MAE: ",mean_absolute_error(y, y_pred))
#Calculamos nuestro MSE
print("MSE: ",mean_squared_error(y, y_pred))
#Calculamos nuestro R2
print("R2: ",r2_score(y, y_pred))
#Calculamos nuestro pearson
print("Pearson: ",pearsonr(y, y_pred)[0])
#Calculamos nuestro spearman
print('Spearman: ',spearmanr(y, y_pred)[0])

MAE:  0.09954902076950754
MSE:  0.016595516631498098
R2:  0.06136779532871073
Pearson:  0.24772525076462787
Spearman:  0.2680704390216584


### Para Multi

In [14]:
#Partimos
X = data_multi_train_prep.drop(['complexity','id','corpus'],axis=1)
y = data_multi_train_prep['complexity']

In [15]:
#Generamos el modelo
regression_multi = Ridge(alpha=1e-05, normalize=True).fit(X,y)

In [16]:
#Realizamos la predicción
y_pred = regression_multi.predict(X)

In [17]:
#Almacenamos el dataset
data_pred = pd.DataFrame({'Y':y,'Y_pred':y_pred})
data_pred.to_csv('Predictions/predictions_data_multi_train.csv')

In [18]:
#Calculamos nuestro MAE
print("MAE: ",mean_absolute_error(y, y_pred))
#Calculamos nuestro MSE
print("MSE: ",mean_squared_error(y, y_pred))
#Calculamos nuestro R2
print("R2: ",r2_score(y, y_pred))
#Calculamos nuestro pearson
print("Pearson: ",pearsonr(y, y_pred)[0])
#Calculamos nuestro spearman
print('Spearman: ',spearmanr(y, y_pred)[0])

MAE:  0.1120971933455431
MSE:  0.019476565854190407
R2:  0.19413899990769612
Pearson:  0.44061217433847033
Spearman:  0.43796390682488073


---
# Realizando predicciones

### Para single

In [19]:
#---------------------
# Trial
# --------------------
#Partimos
X = data_single_trial_prep.drop(['complexity','id','subcorpus'],axis=1)
y = data_single_trial_prep['complexity']

#Realizamos la predicción
y_pred = regression_single.predict(X)

#Calculamos nuestro MAE
print("MAE: ",mean_absolute_error(y, y_pred))
#Calculamos nuestro MSE
print("MSE: ",mean_squared_error(y, y_pred))
#Calculamos nuestro R2
print("R2: ",r2_score(y, y_pred))
#Calculamos nuestro pearson
print("Pearson: ",pearsonr(y, y_pred)[0])
#Calculamos nuestro spearman
print('Spearman: ',spearmanr(y, y_pred)[0])

#Almacenamos el dataset
data_pred = pd.DataFrame({'Y':y,'Y_pred':y_pred})
data_pred.to_csv('Predictions/predictions_data_single_trial.csv')

MAE:  0.0994725999119254
MSE:  0.017131666274555438
R2:  0.09326841862355062
Pearson:  0.31917538324494854
Spearman:  0.32520881395810947


In [20]:
#---------------------
# Test
# --------------------
#Partimos
X = data_single_test_prep.drop(['id','corpus'],axis=1)

#Realizamos la predicción
y_pred = regression_single.predict(X)

#Almacenamos el dataset
data_pred = pd.DataFrame({'Y_pred':y_pred})
data_pred.to_csv('Predictions/predictions_data_single_test.csv')
data_pred.head()

,Y_pred
0,0.254448
1,0.334996
2,0.248730
3,0.300707
4,0.275775


### Para multi

In [21]:
#---------------------
# Trial
# --------------------
#Partimos
X = data_multi_trial_prep.drop(['complexity','id','subcorpus'],axis=1)
y = data_multi_trial_prep['complexity']

#Realizamos la predicción
y_pred = regression_multi.predict(X)

#Calculamos nuestro MAE
print("MAE: ",mean_absolute_error(y, y_pred))
#Calculamos nuestro MSE
print("MSE: ",mean_squared_error(y, y_pred))
#Calculamos nuestro R2
print("R2: ",r2_score(y, y_pred))
#Calculamos nuestro pearson
print("Pearson: ",pearsonr(y, y_pred)[0])
#Calculamos nuestro spearman
print('Spearman: ',spearmanr(y, y_pred)[0])

#Almacenamos el dataset
data_pred = pd.DataFrame({'Y':y,'Y_pred':y_pred})
data_pred.to_csv('Predictions/predictions_data_multi_trial.csv')

MAE:  0.10958111008070552
MSE:  0.018811530788258764
R2:  0.19611811058487505
Pearson:  0.45412137392632423
Spearman:  0.47076213343918616


In [22]:
#---------------------
# Test
# --------------------
#Partimos
X = data_multi_test_prep.drop(['id','corpus'],axis=1)

#Realizamos la predicción
y_pred = regression_multi.predict(X)

#Almacenamos el dataset
data_pred = pd.DataFrame({'Y_pred':y_pred})
data_pred.to_csv('Predictions/predictions_data_multi_test.csv')
data_pred.head()

,Y_pred
0,0.329599
1,0.462949
2,0.378562
3,0.385867
4,0.405812
